## MAC0318 - Teoria de controle

## Representação de estado

Nesse tutorial revisamos alguns conceitos relacionados a sistemas referenciais e mudanças de coordenadas.

O *estado* descreve matematicamente a configuração do robô em um determinado instante. Ele pode incluir informação sobre a localização do robô assim como fatores relativos ao ambiente que afetem o robô (por exemplo, a temperatura do ambiente, ou o valor de sensores). A escolha do estado depende da tarefa que queremos resolver, assim como do domínio de atuação.

Idealmente, gostaríamos que o estado possuisse as seguintes propriedades:

1. *Propriedade de Markov*: O estado futuro independente do passado dado o presente


$$
x_{t+1} = f(x_t, x_{t-1}, \dots, x_0; u_t, \dots, u_0) = f(x_t; u_t)
$$


2. *Suficência e eficiência estatística*: O estado é um estatística suficiente minimal, ou seja, contém todas e apenas informações relevantes para resolução da tarefa pretendida
3. *Tratabilidade*: o estado permite calcular eficientemente o comportamento do robô
4. *Generalibilidade*: o estado é robusto a pequenas mudanças no domínio e na tarefa

## Pose

O exemplo mais comum de estado em robótica é a *pose* ou *postura* do robô.

A *pose* contém a posição do robô no ambiente (relativa a um referencial) assim como sua orientação. Para um robô que atue apenas no solo, como é nosso caso nessa disciplina, ignorando eventuais diferenças de altitude do solo, podemos assumir que a pose é dada pelas coordenadas do centro de referencial local do robô $(x,y)$ e do ângulo $\theta$ de rotação do referencial em relação a um referencial global fixo:

$$
q = 
\begin{bmatrix}
x & y & \theta
\end{bmatrix}
$$

Outra forma de representar a pose é considerar a transformação linear homogênea em relação a uma posição inicial (em relação a algum referencial), formada por uma translação $t$ seguida por uma rotação $R$:

$$
T = 
\begin{bmatrix}
R & t \\
0 & 1
\end{bmatrix}
=
\begin{bmatrix}
\cos(\theta) & -\sin(\theta) & x \\
\sin(\theta) & \cos(\theta) & y \\
0 & 0 & 1
\end{bmatrix}
$$
onde $R$ é a matriz de rotação
$$
R = 
\begin{bmatrix}
\cos(\theta) & -\sin(\theta) \\
\sin(\theta) & \cos(\theta) \\
\end{bmatrix}
$$
e $t=\begin{bmatrix}
x &
y 
\end{bmatrix}^t$ é o vetor de translação representando o centro do referencial local do robô no referencial global.

A matrix $T$ faz parte do chamado [grupo](https://en.wikipedia.org/wiki/Euclidean_group) (no sentido algébrico) especial euclideano de dimensão 2, denominado $SE(2)$, que representa as simetrias do espaço euclideano que preservam distância entre pontos e orientação, ou seja, rotação e translação.

Uma propriedade importante da matriz de transformação $T$ é que
$$
T^{-1} = 
\begin{bmatrix}
R^T & -R^Tt \\
0 & 1
\end{bmatrix} .
$$

In [91]:
# Alguns comandos úteis em numpy
import numpy as np
# Criando um vetor
r = np.array([ 3.0, 4.0 ])
print('r =', r)
# Criando uma matriz 2x2
R = np.array([ 
                [np.cos(0.5), -np.sin(0.5)],
                [np.sin(0.5),  np.cos(0.5)]
             ])
print('R =\n', R)
print(R.ndim, R.shape)
# Compondo matriz de blocos
T = np.block([
                [R, r.reshape((2,1))], 
                [np.zeros((1, 2)), np.array([1])]
              ])
print('T =\n', T)
# Transposta de uma matriz
Rt = R.T
print('Rt =\n', Rt)
# Inversa de matriz em SE(2)
Tinv = np.block([
                 [Rt, -Rt @ r.reshape((2,1))],
                 [np.zeros((1,2)), np.array([1])]
                ])
print('T^-1 =', Tinv)
# Multiplicação de matrizes
I = T @ Tinv # multiplicação de matrizes
print( I )

r = [3. 4.]
R =
 [[ 0.87758256 -0.47942554]
 [ 0.47942554  0.87758256]]
2 (2, 2)
T =
 [[ 0.87758256 -0.47942554  3.        ]
 [ 0.47942554  0.87758256  4.        ]
 [ 0.          0.          1.        ]]
Rt =
 [[ 0.87758256  0.47942554]
 [-0.47942554  0.87758256]]
T^-1 = [[ 0.87758256  0.47942554 -4.55044984]
 [-0.47942554  0.87758256 -2.07205363]
 [ 0.          0.          1.        ]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


## Exercício 1

<figure>
  <div style="text-align:center;">
  <img src="img/pose_exercise.png">
  </div>
</figure>

A figura acima ilustra a posição do robô (ponto laranja) e sua orientação ($\theta$), dada pelo ângulo entre o eixo $x$ no sentido antihorário.

1. Escreva o vetor $q \in \mathbb{R}^3$ representando a pose do robô no referencial global $W$

In [1]:
theta = np.deg2rad(0) # converte de graus para radianos = (np.pi * 60) / 180.

q_W = np.array([0, 0, theta]) # complete com solução

print(q_W)

[0. 0. 0.]


2. Escreva o vetor $q \in \mathbb{R}^3$ representando a pose do robô no referencial local (do robô) $R$

In [2]:
q_R = np.array([0, 0, 0]) # complete com solução

print(q_R)

[0 0 0]


3. Escreva a matriz $T \in SE(2) \subseteq \mathbb{R}^{3 \times 3}$ representando a pose do robô relativa ao referencial global

In [10]:
T = np.array([
    [np.cos(theta), -np.sin(theta), q_g[0]],
    [np.sin(theta),  np.cos(theta), q_g[1]],
    [            0,              0, 1]
])

print(T) # Esperado: [[ 0.5       -0.8660254  2.       ]
         #            [ 0.8660254  0.5        3.       ]
         #            [ 0.         0.         1.       ]]

[[ 0.5       -0.8660254  2.       ]
 [ 0.8660254  0.5        3.       ]
 [ 0.         0.         1.       ]]


# Mudança de referencial

A representação como matriz em $SE(n)$ é conveniente pois permite facilmente que a descrição da pose em um referencial seja reescrita com relação a outro referencial. 

Por exemplo, se sabemos a pose $T_A^O$ no referencial de origem $O$ e sabemos a pose $T_B^A$ no referencial $A$, obtemos a pose $T_B^O$ no referencial de origem como
$$
T_B^O = T_A^O T_B^A .
$$

Outra transformação útil entre referenciais é
$$
T_A^B = (T_B^A)^{-1} .
$$


## Exercício 2

Na figura abaixo, o ponto laranja representa o robô (uma pose) e o ponto azul representa o obstáculo (um ponto). As medidas dos eixos são em metros.

<figure>
  <div style="text-align:center;">
  <img src="img/moving_frame_exercise_1.png">
  </div>
</figure>

Sabendo que a pose do robô no referencial global é dada por
$$
q_r^W = \begin{bmatrix} 2 & 0.4 & 110 \end{bmatrix} 
$$
e que a distância e o ângulo entre o robô e obstáculo são de $0{,}3$m e $50$ graus, respectivamente, determine a posição do obstáculo no referencial global.

In [ ]:
# Escreva sua solução


print( T_O_W ) 

In [10]:
# Complete com sua solução (posição do obstáculo no referencial global)
p_o_W = np.array([T_O_W[0,2], T_O_W[1,2]])    
print(p_o_W) # resultado esperado [1.71809221, 0.50260604, 160.]

[0. 0.]


## Exercício 3

Na figura abaixo, a pose do robô no referencial global é  $q^W = \begin{bmatrix} 3.5 & -1.2 & 45 \end{bmatrix}$.

<figure>
  <div style="text-align:center;">
  <img src="img/moving_frame_exercise_2.png">
  </div>
</figure>

Determine a posição do obstáculo (ponto azul) na figura abaixo no referencial local do robô.

In [ ]:
p_R_W = np.array([3.5, -1.2]) # posição do robô no referencial global
theta_R_W = np.deg2rad(45)    # ângulo do robô no referencial global

p_o_W = np.array([4.0, -1.0]) # posição do obstáculo no referencial global

# Escreva sua solução


p_o_R = np.array([ T_o_R[0,2], T_o_R[1,2] ])

print(p_o_R) # resultado esperado [ 0.49497475 -0.21213203]

In [14]:
# Solução alternativa

T_W_R @ np.block([p_o_W, 1])

array([ 0.49497475, -0.21213203,  1.        ])